In [ ]:
from pyalgotrade import strategy
from pyalgotrade.bitstamp import barfeed
from pyalgotrade.bitstamp import broker
from pyalgotrade.twitter import feed as twitterfeed

class Strategy(strategy.BaseStrategy):
    def __init__(self, feed, brk, twitterFeed):
        super(Strategy, self).__init__(feed, brk)
        self.__instrument = "BTC"

        # Subscribe to Twitter events.
        twitterFeed.subscribe(self.__onTweet)

    def __onTweet(self, data):
        try:
            self.info("Twitter: %s" % (data["text"]))
        except KeyError:
            pass

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info("Price: %s." % bar.getClose())

In [ ]:
consumer_key = ""
consumer_secret = ""

access_token = ""
access_token_secret = ""

track = ["bitcoin", "btc", "bitstamp"]
follow = []
languages = ["en"]
twitterFeed = twitterfeed.TwitterFeed(consumer_key, 
                                      consumer_secret, 
                                      access_token, 
                                      access_token_secret, 
                                      track, 
                                      follow, 
                                      languages)

barFeed = barfeed.LiveTradeFeed()
brk = broker.PaperTradingBroker(1000, barFeed)
strat = Strategy(barFeed, brk, twitterFeed)

# Add twitterFeed to the event dispatch loop.
strat.getDispatcher().addSubject(twitterFeed)
strat.run()